In [3]:
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import load_tr_cities, translation
warnings.filterwarnings('ignore')

In [99]:
def load_work_experiences(path: str, size: int = 10, seperator: str = 'exp') -> pd.DataFrame:

    df_ = pd.read_csv(path)
    df_['start_date'] = pd.to_datetime(df_['start_year_month'].apply(lambda x: '-'.join([str(x)[:4], str(x)[4:]])))
    df_ = df_.drop(columns = ['start_year_month'], axis = 1)

    df_.loc[df_['location'] =='Intern', 'location'] = np.nan
    df_.loc[df_['location'] =='Stajyer', 'location'] = np.nan
    df_.loc[df_['location'] =='Sales Consultant', 'location'] = np.nan
    df_.loc[df_['location'] =='asistan', 'location'] = np.nan
    df_.loc[df_['location'] =='Yazılım stajyeri', 'location'] = np.nan
    df_.loc[df_['location'] =='stajer mühendis', 'location'] = np.nan
    df_.loc[df_['location'] =='stajyer mühendis', 'location'] = np.nan
    df_.loc[df_['location'] =='Yazılım stajı', 'location'] = np.nan
    df_.loc[df_['location'] =='Donanım stajı', 'location'] = np.nan
    df_.loc[df_['location'] =='staj', 'location'] = np.nan    
    df_.loc[df_['location'] =='stajer', 'location'] = np.nan
    df_.loc[df_['location'] =='stajyer', 'location'] = np.nan
    df_.loc[df_['location'] =='User support application specialist', 'location'] = np.nan
    df_.loc[df_['location'] =='front-end developer', 'location'] = np.nan
    df_.loc[df_['location'] =='Kasa Personeli', 'location'] = np.nan
    df_.loc[df_['location'] =='web developer', 'location'] = np.nan
    df_.loc[df_['location'] =='Software development', 'location'] = np.nan
    df_.loc[df_['location'] =='Software developer', 'location'] = np.nan
    df_.loc[df_['location'] =='software developer', 'location'] = np.nan
    df_.loc[df_['location'] =='Visual Studio Asp.Net Developer', 'location'] = np.nan
    df_.loc[df_['location'] == 'yazılım mühendisi', 'location'] = np.nan
    df_.loc[df_['location'] == 'Supervisor chief engineer', 'location'] = np.nan
    df_.loc[df_['location'] == 'Computer Engineer (System Administrator-Specialist)', 'location'] = np.nan
    df_.loc[df_['location'] == 'Civil Engineer', 'location'] = np.nan
    df_.loc[df_['location'] == 'Öğrenci stajeri', 'location'] = np.nan
    df_.loc[df_['location'] == 'İzmir Yüksek Teknoloji Enstitüsü Teknopark', 'location'] = 'Izmir, Turkey'
    df_.loc[df_['location'] == 'İstanbul/Avrupa', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Sakarya Arifiye', 'location'] = 'Sakarya, Turkey'
    df_.loc[df_['location'] == 'Kepez, Çanakkale, Türkiye', 'location'] = 'Canakkale, Turkey'
    df_.loc[df_['location'] == 'Ataşehir', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'atasehir', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Muğla milas', 'location'] = 'Mugla, Turkey'
    df_.loc[df_['location'] == 'Besiktas', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Istanbu', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ataşehir', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'İçerenköy Mh. Karaman Çiftlik Yolu Cd. Firuze Sk. Secretcv Plaza No:2 Ataşehir', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Kahraman Maras, Turkey', 'location'] = 'Kahramanmaras, Turkey'
    df_.loc[df_['location'] == 'Pendik', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Pendik Teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Levent', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Levent,Davutpaşa', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Levent, Kolektif House', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Kolektif House | Levent', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Samandıra/Kadıköy/Levent', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beşiktaş', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beşiktaş/Akatlar', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Fulya Beşiktaş', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ulus/Beşiktaş', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beşiktaş Barbaros', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'İçerenköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'İçerenköy Üner Plaza', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Kağıthane', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Bostancı', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ümraniye', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ümraniye ', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Netaş HQ Ümraniye', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Mecidiyeköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Sarıyer', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Şişli', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Esentepe Şişli İst', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Esentepe / Sisli', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Kadıköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beşiktaş - Kadıköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'beşiktaş', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Avcılar', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'avcılar', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Avcılar Teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'pendik', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beyoğlu', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beyoğlu, Galata', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Karaköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'bostancı ebebek', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'bostancı ', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == '4.levent', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'ista', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'ist', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'İst', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Çekmeköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'çekmeköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'nişantaşı', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'ataşehir', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Batı ataşehir', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Taksim', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Kadıköy/Fenerbahçe', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Göztepe', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ortaköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız teknik üniversitesi teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'yıldız teknik üniversitesi teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'ytü teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Davutpaşa YTU teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız teknopark ', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'ytu davutpasa teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Gebze', 'location'] = 'Kocaeli, Turkey'
    df_.loc[df_['location'] == 'Maslak', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'İTÜ Maslak Kampüsü', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Maslak, Turkey', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Acıbadem', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Acıbadem genel müdürlük', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Acıbadem/ Anadolu Yakası Bölge Müdürlüğü', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Üsküdar', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Dudullu', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Dudullu ', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Dudullu OSB 2. Cadde No:4', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Buyaka Tower-2, Ümraniye / İSTANBUL', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ümraniye / Turkey', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ümraniye,Turkey', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yukarı Dudullu- Ümraniye / İSTANBUL', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Ümraniye / Gebze', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Adres: Fatih Sultan Mehmet. Mah. Balkan Cad. No: 47 Casper Plaza, Ümraniye / İst', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yukarı Dudullu Organize Sanayi Bölgesi,  And Sitesi, No:18 34775 Ümraniye / İST.', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'kadıköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'moda, kadıköy', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'dudullu', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'YTÜ Beşiktaş / İSTANBUL', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beşiktaş, Türkiye', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Beştepe Mah., Ankara, Türkiye', 'location'] = 'Ankara, Turkey'
    df_.loc[df_['location'] == 'Beşiktaş ', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Kemalpaşa', 'location'] = 'Izmir, Turkey'
    df_.loc[df_['location'] == 'Kemalpaşa Organize Sanayi Bölgesi - ITOB', 'location'] = 'Izmir, Turkey'
    df_.loc[df_['location'] == 'Göztepe, Bağdat Caddesi', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Göztepe Yerleşkesi', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Göztepe Kampüsü', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Marmara University, Göztepe', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Marmara Üniversitesi', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız Teknik Üniversitesi', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız Teknik Üniversitesi Teknopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız Technical University Technopark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'İstanbul Technical University - Technocity', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yeditepe University', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yildiz Technical University R&D Lab', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Istanbul University-Cerrahpasa, Engineering Faculty', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Teknopark Yıldız Teknik Üniversitesi', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız Teknik University', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız Teknik Üniversitesi TeknoPark', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Istanbul Technical University IT Head Office ', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'İstanbul Üniversitesi - Cerrahpaşa Mühendislik Fakültesi', 'location'] = 'Istanbul, Turkey'
    df_.loc[df_['location'] == 'Yıldız Teknik Üniversitesi - Teknopark', 'location'] = 'Istanbul, Turkey'

    df_ = df_.loc[df_['start_date'].dt.year != 2019].sort_values(by = ['user_id', 'start_date']).reset_index(drop = True)

    exp_df = df_.groupby('user_id').agg(total_experience = ('start_date', lambda x: int(str(pd.to_datetime('2019-01-01') - x.min()).split()[0])),
                                        last_experience = ('start_date', lambda x: int(str(pd.to_datetime('2019-01-01') - x.max()).split()[0]))
    ).merge(df_.groupby('user_id').agg(unique_company = ('company_id', 'nunique')), on = ['user_id'], how = 'left'
    ).reset_index().assign(avg_days_to_quit = lambda x: x.total_experience / x.unique_company)
    
    #last experience eklenecek !!! (2019 - son işe girişi)



    #df_['location'] = df_['location'].apply(lambda x: str(x).replace('Türkiye', 'Turkey'))
    #df_['location'] = df_['location'].apply(lambda x: x.upper().strip())
    #df_['location'] = df_['location'].apply(lambda x: translation(str(x)))
    
    #tr_cities = load_tr_cities()
    #for city in tr_cities:
    #    df_['location'] = df_['location'].apply(lambda x: city if city in x else x)
#
    #df_.loc[df_['location'] == 'NAN', 'location'] = np.nan
    #df_ = df_.sort_values(by=['user_id', 'start_date'])
    #df_ = df_.drop_duplicates(subset=['user_id', 'start_date']).reset_index(drop=True)
#
    #most_freq_locs = df_['location'].value_counts().keys()[:size].tolist()
    #grouped = df_.groupby(by='user_id', as_index=False).agg({'location': 'unique'})
    #for lang in tqdm(most_freq_locs):
    #    grouped[f'{seperator}_{lang}'] = grouped['location'].apply(lambda x: 1 if lang in x else 0)

    #grouped = grouped.merge(df_.groupby(by='user_id', as_index=False).agg(total_locations=('location', 'nunique')), on=['user_id'], how='left')
    #grouped = grouped.drop(columns=['location'], axis=1)
    
    return exp_df


In [100]:
#df = load_work_experiences('../../../datasets/garanti-bbva-data-camp/work_experiences.csv')

#df.head(20)

,user_id,total_experience,last_experience,unique_company,avg_days_to_quit
0,0,4870,4870,1,4870.000000
1,2,761,214,3,253.666667
2,5,579,579,1,579.000000
3,7,914,31,2,457.000000
4,10,1979,245,2,989.500000
5,11,518,153,2,259.000000
6,12,2010,61,4,502.500000
7,13,2863,306,2,1431.500000
8,14,1767,1767,1,1767.000000
9,16,579,579,1,579.000000


In [96]:
#df.groupby('user_id').agg(total_experience = ('start_date', lambda x: int(str(pd.to_datetime('2019-01-01') - x.min()).split()[0]))).merge(
#
#    df.groupby('user_id').agg(unique_company = ('company_id', 'nunique')), on = ['user_id'], how = 'left'
#
#).reset_index().assign(avg_days_to_quit = lambda x: x.total_experience / x.unique_company)

,user_id,total_experience,unique_company,avg_days_to_quit
0,0,4870,1,4870.000000
1,2,761,3,253.666667
2,5,579,1,579.000000
3,7,914,2,457.000000
4,10,1979,2,989.500000
...,...,...,...,...
52997,66269,5388,1,5388.000000
52998,66270,1522,1,1522.000000
52999,66271,6270,7,895.714286
53000,66272,2375,5,475.000000


In [66]:
#df.loc[df['location'].astype(str).str.contains('tekno'), 'location'].value_counts()[:20]

odtü teknokent                                                5
Hacettepe teknokent                                           4
teknopark istanbul                                            2
Nanoteknoloji Alanında Finalist                               2
Odtü teknokent                                                2
odtu teknokent                                                1
teknopark                                                     1
Tubitak Gebze Yerleşkesi, teknoloji serbest bölgesi A Blok    1
hacettepe teknokent                                           1
istanbul teknopark                                            1
Bilkent teknokent                                             1
İtü arı teknokent                                             1
Kayseri teknopark                                             1
ankara ivedik teknopark                                       1
Ankara, ODTU teknokent                                        1
İstanbul teknopark                      

In [43]:
#df.loc[(df['location'].astype(str).str.contains('Kemalpaşa')) & (~df['location'].astype(str).str.contains('aa')), 'location'].value_counts().keys().tolist()

['Kemalpaşa, İzmir, Türkiye',
 'Kemalpaşa Mah., İzmir, Türkiye',
 'Kemalpaşa Mah., İzmir Province, Turkey',
 'Kemalpaşa, İzmir',
 'Kemalpaşa/İzmir',
 'Kemalpaşa / İzmir',
 'Kemalpaşa OSB Mahallesi İzmir-Ankara Asfaltı 25.km No.68 Kemalpaşa-İZMİR']